In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.output_parsers.string import StrOutputParser
import chromadb
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
os.environ["OPENAI_API_BASE"] = "your_api_base"
os.environ["OPENAI_API_KEY"] = "your_api_key"

In [ ]:
pdf_folder_path = "your_pdf_folder_path"
documents = []
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)
client = chromadb.Client()
if client.list_collections():
    consent_collection = client.create_collection("mentor_collection")
else:
    print("Collection already exists")
vectordb = Chroma.from_documents(
    documents=chunked_documents,
    embedding=OpenAIEmbeddings(),
    persist_directory="./chroma_store"
)
vectordb.persist()

In [ ]:
loader.load()

In [ ]:
 #"the question. The question is regarding the ability of the mentor to research"
 #   "specific field."

from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer "
    "the question. The question is regarding the ability of the mentor to research"
    "specific field."
    "You response should contain mentor_ID like '746', '518',you can get the id from content source"
    "If you don't know the answer, say that you don't know"
    "\n\n"
    "{context}")

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
)

#question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
retriever = vectordb.as_retriever()

chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("recommend an engineering mentor")
#rag_chain = create_retrieval_chain(retriever, question_answer_chain) 

In [11]:
## build a CSV retriever
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader

csv_loader = CSVLoader(file_path='your_csv_file_path')
documents = csv_loader.load()
embeddings=OpenAIEmbeddings()

csv_directory = './csv_db'
csv_store = Chroma.from_documents(documents, embeddings, csv_directory)

In [ ]:
# 通过人工链接将上一个链的信息传递到新的链
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)
csv_system_prompt = (
    "You are an assistant for question-answering tasks."
    "Based on the human input, you can get the research mentor ID"
    "Use the following pieces of retrieved context to present the mentor ID's feedback and available time."
    "\n\n"
    "{context}")

csv_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", csv_system_prompt),
            ("human", "{input}"),
        ]
)

csv_retriever = csv_store.as_retriever()

csv_chain = (
    {"context": csv_retriever, "input": RunnablePassthrough()}
    | csv_prompt
    | llm
    | StrOutputParser()
)

csv_chain.invoke("I recommend mentor with ID 518, Hu, who has experience in mentoring undergraduate students in the field of Mechanical and Aerospace Engineering with a specialization in MEMS.")

In [13]:
final_chain = chain | csv_chain

In [ ]:
final_chain.invoke("recommend a CS mentor")